In [2]:
import altair as alt
import pandas as pd

# CUDA vs. OpenCL performance analysis example

First, generate parsed log dataset using

```bash
parse_fah_log dataframe --data-dir ../data --output ../data/benchmark_data PROJ17101 PROJ17102
```

In [4]:
data = pd.read_feather("./checkpoint_17110").set_index(["run", "clone", "gen"]).sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2590 entries, (0, 0, 0) to (18, 33, 0)
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   os                     2590 non-null   object 
 1   platform_name          2590 non-null   object 
 2   platform_vendor        2590 non-null   object 
 3   platform_version       2590 non-null   object 
 4   device_name            2590 non-null   object 
 5   device_vendor          2590 non-null   object 
 6   device_version         2590 non-null   object 
 7   device_driver_version  2590 non-null   object 
 8   cuda_enabled           2590 non-null   bool   
 9   perf_ns_per_day        2588 non-null   float64
dtypes: bool(1), float64(1), object(8)
memory usage: 205.4+ KB


In [10]:
device_name = "GeForce RTX 2080 Ti"

df = (data
      .pipe(lambda df: df[df["device_name"] == device_name])
      .assign(platform=lambda df: df["cuda_enabled"].replace({False: "CUDA", True: "OpenCL"}))).reset_index()

bars = alt.Chart().mark_bar().encode(
    x="platform:N",
    y=alt.Y("mean(perf_ns_per_day):Q", title="mean ns/day"),
    color='platform:N',
)

error_bars = alt.Chart().mark_errorbar(extent='ci').encode(
    x=alt.X("platform:N", title=None),
    y=alt.Y("perf_ns_per_day:Q", title=None)
)

alt.layer(bars, error_bars, data=df).facet(
    column="run:O",
    title=device_name
)

alt.FacetChart(...)